# 使用表单识别器分析收据

![拿着收据的机器人](./images/receipt_analysis.jpg)

在计算机视觉的人工智能 (AI) 领域中，光学字符识别 (OCR) 通常用于读取印刷体文档或手写文档。通常只需从文档中提取文本，并将其处理为可供进一步处理或分析的格式。

更复杂的 OCR 场景是从表单（如采购订单或发票）中提取信息，并从语义上理解表单中各字段表示的意思。**表单识别器**服务专为此类 AI 问题而设计。

## 查看收据

本例将使用表单识别器的内置模型来分析收据。

单击下面的“**运行单元格**”(&#9655;) 按钮（位于单元格左侧）以运行它，并查看将使用表单识别器来进行分析的收据示例。

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Load and display a receipt image
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## 创建表单识别器资源

首先在 Azure 订阅中创建表单识别器资源：

1. 在另一个浏览器标签页中，打开 Azure 门户 (https://portal.azure.com) 并使用 Microsoft 帐户登录。
2. 选择“**+ 创建资源**”并搜索“*表单识别器*”。
3. 在服务列表中，选择“**表单识别器**”。
4. 在“**表单识别器**”边栏选项卡中选择“**创建**”。
5. 在“**创建**”边栏选项卡中，输入以下详细信息，然后选择“**创建**”
   * **名称**：服务的唯一名称
   * **订阅**：你的 Azure 订阅
   * **区域**：任何可用区域
   * **定价层**：F0
   * **资源组**：之前使用的现有资源组
   * **我确认我已阅读并理解以下通知**：已选中。
6. 等待服务创建完毕。
7. 在 Azure 门户中查看新建的表单识别器服务，并在“**密钥和终结点**”页面上复制“**Key1**”和**终结点**值，然后将这两个值粘贴到下方的代码单元格中，替换“**YOUR_FORM_KEY**”和“**YOUR_FORM_ENDPOINT**”。

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## 分析收据

现在可以使用表单识别器来分析收据了。

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Create a client for the form recognizer servicer
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Submit the file data to form recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Get the results
    receipt_data = analyze_receipt.result()

    # Print the extracted data for the first (and only) receipt
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


请注意，表单识别器能够解释表单中的数据，正确识别商家地址和电话号码、交易日期和时间以及单项商品、小计、税款和总额。

## 更多信息

有关表单识别器服务的详细信息，请参阅[表单识别器文档](https://docs.microsoft.com/zh-cn/azure/cognitive-services/form-recognizer/index)